In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.auto import tqdm

In [16]:
user_train = np.load('../data/user_train.npy', allow_pickle=True).item()

In [17]:
seq = user_train['sequences']['01FJRKCP4GE1W1DFX51C']['keypoints']
seq.shape

(4500, 11, 24, 2)

In [18]:
np.isnan(seq).sum()

432000

In [19]:
seq_norm = seq.copy()
seq_norm[np.isnan(seq_norm)] = 0
seq_len, num_flies, _, _ = seq_norm.shape
seq_norm = seq_norm.reshape(seq_len, num_flies, -1)
seq_norm = torch.Tensor(seq_norm)
seq_norm.shape

torch.Size([4500, 11, 48])

In [20]:
torch.isnan(seq_norm).sum()

tensor(0)

In [72]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=48, nhead=8)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.head = nn.Linear(48, 48)
        
    def forward(self, x):
        bs, seq, dim = x.shape
        x = self.encoder(x)
        x = x.reshape(-1, dim)
        x = self.head(x)
        x = x.reshape(bs, seq, -1)
        return x

In [84]:
DEVICE = 'cuda'
mask_prob = 0.1
torch.cuda.empty_cache()

In [85]:
model = Model()

In [88]:
bs = 64

model = model.to(DEVICE).train()
seq_norm = seq_norm.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=0.002)

batches = torch.chunk(seq_norm, chunks=seq_norm.size(0) // (bs - 1), dim=0)

outs = []

ebar = tqdm(range(100))
for e in ebar:
    bbar = tqdm(batches)
    for batch in bbar:
        mask = torch.rand_like(batch) > mask_prob
        masked_batch = batch * mask
        out = model(masked_batch)
        loss = F.mse_loss(out, batch)

        optim.zero_grad()
        loss.backward()
        optim.step()

        bbar.set_description(f'Loss: {loss.item():0.5}')
        ebar.set_description(f'Loss: {loss.item():0.5}')
    
    outs.append(out)
outs = torch.cat(outs)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

...

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]